In [3]:
from google.colab import files
src = list(files.upload().values())[0]
open('mylib.py','wb').write(src)
import mylib

Saving prepare_data.py to prepare_data.py


In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, Input, BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, Adam
import prepare_data as pd
import numpy
import math

In [5]:
from google.colab import files
uploaded = files.upload()

Saving baby_GT.bmp to baby_GT (1).bmp


In [0]:
def psnr(target, ref):
    target_data = numpy.array(target, dtype=float)
    ref_data = numpy.array(ref, dtype=float)
    diff = ref_data - target_data
    diff = diff.flatten('C')
    rmse = math.sqrt(numpy.mean(diff ** 2.))
    return 20 * math.log10(255. / rmse)

In [0]:
def model():
    SRCNN = Sequential()
    C1 = SRCNN.add(Conv2D(nb_filter=128, nb_row=9, nb_col=9, init='glorot_uniform',
                     activation='relu', border_mode='valid', bias=True, input_shape=(32, 32, 1)))
    C2 = SRCNN.add(Conv2D(nb_filter=64, nb_row=3, nb_col=3, init='glorot_uniform',
                     activation='relu', border_mode='same', bias=True))
    C3 = SRCNN.add(Conv2D(nb_filter=1, nb_row=5, nb_col=5, init='glorot_uniform',
                     activation='linear', border_mode='valid', bias=True))
    adam = Adam(lr=0.0003)
    SRCNN.compile(optimizer=adam, loss='mean_squared_error', metrics=['mean_squared_error'])
    return SRCNN

In [0]:
def predict_model():
    SRCNN = Sequential()
    C1 = SRCNN.add(Conv2D(nb_filter=128, nb_row=9, nb_col=9, init='glorot_uniform',
                     activation='relu', border_mode='valid', bias=True, input_shape=(None, None, 1)))
    C2 = SRCNN.add(Conv2D(nb_filter=64, nb_row=3, nb_col=3, init='glorot_uniform',
                     activation='relu', border_mode='same', bias=True))
    C3 = SRCNN.add(Conv2D(nb_filter=1, nb_row=5, nb_col=5, init='glorot_uniform',
                     activation='linear', border_mode='valid', bias=True))
    adam = Adam(lr=0.0003)
    SRCNN.compile(optimizer=adam, loss='mean_squared_error', metrics=['mean_squared_error'])
    return SRCNN

In [10]:
from google.colab import files
src = list(files.upload().values())[0]
open('mylib.py','wb').write(src)
import mylib

Saving train_scale_4.h5 to train_scale_4.h5


In [15]:
from google.colab import files
src = list(files.upload().values())[0]
open('mylib.py','wb').write(src)
import mylib

Saving test_scale_4.h5 to test_scale_4 (1).h5


In [26]:
def train():
    srcnn_model = model()
    print(srcnn_model.summary())
    data, label = pd.read_training_data("train_scale_4.h5")
    val_data, val_label = pd.read_training_data("test_scale_4.h5")

    checkpoint = ModelCheckpoint("NIAZ.h5", monitor='val_loss', verbose=1, save_best_only=True,
                                 save_weights_only=False, mode='min')
    
    callbacks_list = [checkpoint]

    srcnn_model.fit(data, label, batch_size=128, validation_data=(val_data, val_label),
                    callbacks=callbacks_list, shuffle=True, nb_epoch=1, verbose=1)
    
def predict():
      srcnn_model = predict_model()
      srcnn_model.load_weights("NIAZ.h5")
      IMG_NAME = "baby_GT.bmp"
      INPUT_NAME = "input2.jpg"
      OUTPUT_NAME = "pre2.jpg"
      
      import cv2
      img = cv2.imread(IMG_NAME, cv2.IMREAD_COLOR)
      img = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
      shape = img.shape
      Y_img = cv2.resize(img[:, :, 0], (shape[1] // 2, shape[0] // 2), cv2.INTER_CUBIC)
      Y_img = cv2.resize(Y_img, (shape[1], shape[0]), cv2.INTER_CUBIC)
      img[:, :, 0] = Y_img
      img = cv2.cvtColor(img, cv2.COLOR_YCrCb2BGR)
      cv2.imwrite(INPUT_NAME, img)
      
      Y = numpy.zeros((1, img.shape[0], img.shape[1], 1), dtype=float)
      Y[0, :, :, 0] = Y_img.astype(float) // 255.
      pre = srcnn_model.predict(Y, batch_size=1) * 255.
      pre[pre[:] > 255] = 255
      pre[pre[:] < 0] = 0
      pre = pre.astype(numpy.uint8)
      img = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
      img[6: -6, 6: -6, 0] = pre[0, :, :, 0]
      img = cv2.cvtColor(img, cv2.COLOR_YCrCb2BGR)
      cv2.imwrite(OUTPUT_NAME, img)
      
      # psnr calculation:
      im1 = cv2.imread(IMG_NAME, cv2.IMREAD_COLOR)
      im1 = cv2.cvtColor(im1, cv2.COLOR_BGR2YCrCb)[6: -6, 6: -6, 0]
      im2 = cv2.imread(INPUT_NAME, cv2.IMREAD_COLOR)
      im2 = cv2.cvtColor(im2, cv2.COLOR_BGR2YCrCb)[6: -6, 6: -6, 0]
      im3 = cv2.imread(OUTPUT_NAME, cv2.IMREAD_COLOR)
      im3 = cv2.cvtColor(im3, cv2.COLOR_BGR2YCrCb)[6: -6, 6: -6, 0]
      
      print ("bicubic:")
      print (cv2.PSNR(im1, im2))
      print ("SRCNN:")
      print (cv2.PSNR(im1, im3))


if __name__ == "__main__":
    train()
    predict()
    
files.download('NIAZ.h5')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(activation="relu", input_shape=(32, 32, 1..., kernel_size=(9, 9), filters=128, padding="valid", kernel_initializer="glorot_uniform", use_bias=True)`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(activation="relu", kernel_size=(3, 3), filters=64, padding="same", kernel_initializer="glorot_uniform", use_bias=True)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(activation="linear", kernel_size=(5, 5), filters=1, padding="valid", kernel_initializer="glorot_uniform", use_bias=True)`
  
/usr/local/lib/python3.6/dist-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argum

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 24, 24, 128)       10496     
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 24, 24, 64)        73792     
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 20, 20, 1)         1601      
Total params: 85,889
Trainable params: 85,889
Non-trainable params: 0
_________________________________________________________________
None
Train on 14901 samples, validate on 150 samples
Epoch 1/1
14901/14901 [==============================] - 3s 234us/step - loss: 0.0127 - mean_squared_error: 0.0127 - val_loss: 0.0034 - val_mean_squared_error: 0.0034

Epoch 00001: val_loss improved from inf to 0.00339, saving model to NIAZ.h5


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(activation="relu", input_shape=(None, Non..., kernel_size=(9, 9), filters=128, padding="valid", kernel_initializer="glorot_uniform", use_bias=True)`
  after removing the cwd from sys.path.


bicubic:
34.32213901381487
SRCNN:
4.707760368384115
